# Recommender Systems with Graph Neural Networks in PyG

By Derrick Li, Peter Maldonado, Akram Sbaih as part of the Stanford CS224W course project.

In this tutorial, we implement two GNN recommender system architectures, LightGCN and NGCF, in PyG and apply them to the MovieLens 100K dataset.

________________________
发现这个版本写的好 clean 啊: (LightGCN 写的短，然后有用label encoder, 然后 bipartite 也有处理)
- blog: https://medium.com/stanford-cs224w/recommender-systems-with-gnns-in-pyg-d8301178e377
- code: https://colab.research.google.com/drive/1VQTBxJuty7aLMepjEYE-d7E9kjo51CA1?usp=sharing#scrollTo=oHuXurG8mezC

而且他也是有考虑进去 bipartite graph, 但是没有像我那样按公式硬转， 而是我之前的 offset 写法， 那我看一下他 evaluation 的时候咋写的
```
u_t = torch.LongTensor(train_df.user_id_idx)
i_t = torch.LongTensor(train_df.item_id_idx) + n_users

train_edge_index = torch.stack((
  torch.cat([u_t, i_t]),
  torch.cat([i_t, u_t])
)).to(device)
```

这里还有个类似的，如果上面这个又问题可以做参考对比:
- [Medium][Implementing Neural Graph Collaborative Filtering in PyTorch](https://medium.com/@yusufnoor_88274/implementing-neural-graph-collaborative-filtering-in-pytorch-4d021dff25f3)

----------------
我发现他这里，也是对 training 和 BPR 用的是 bipartite adj mat 的 edge_list

然后对 recall 用的是正常的 interaciton mat 的 edge_list， 那就说明我其实也可以

## Setup

First, we'll install the necessary packages.

Next, let's import all of the modules that we'll use in this notebook.

In [1]:
# Standard library imports
import random
import time

# Third-party imports
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
import torch_geometric
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import degree

from tqdm.notebook import tqdm
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

from torch_sparse import SparseTensor, matmul
from torch_geometric.data import download_url, extract_zip

Lastly, we should double check that our environment is working as expected.

In [2]:
torch_geometric.__version__

'2.1.0'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Dataset and Preprocessing

For this tutorial, we’ll be using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/), which contains 100,000 ratings by 943 users of 1682 items (movies). To ensure the quality of the dataset, each user has rated at least 20 movies. 

We’ll focus on the interactions between users and items, in this case user ratings of movies, but the dataset also provides metadata about users and movies, such as user demographics and movie titles, release dates, and genres. T

The user ratings of movies form a bipartite graph, which we can apply graph machine learning methods to recommend new movies to users.


In [4]:
# url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
# extract_zip(download_url(url, '.'), '.')

movie_path = './ml-latest-small/movies.csv'
rating_path = './ml-latest-small/ratings.csv'
user_path = './ml-latest-small/users.csv'

In [5]:
# rating df 用新版的 
# url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
# Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018
# 这个 density 低很多的
# 原来的 .data 板的 density 很高， 能到 precision / recall 嫩到 (0.2395, 0.3564)
# 看下新的咋样
df = pd.read_csv(rating_path)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [7]:
# columns_name=['user_id','item_id','rating','timestamp']
# df = pd.read_csv("./ml-100k/u.data",sep="\t",names=columns_name)
print(len(df))
display(df.head(5))

100836


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


We only want to use high ratings as interactions in order to predict which movies a user will enjoy watching next.

In [8]:
num_users = len(df['userId'].unique())
num_movies = len(df['movieId'].unique())

print(num_users)
print(num_movies)

610
9724


In [9]:
# What's the distribution of highly rated movies?
print("Rating Distribution")
df.groupby(['rating'])['rating'].count()

Rating Distribution


rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: rating, dtype: int64

In [10]:
# Perform a 80/20 train-test split on the interactions in the dataset
train, test = train_test_split(df.values, test_size=0.2, random_state=16)
train_df = pd.DataFrame(train, columns=df.columns)
test_df = pd.DataFrame(test, columns=df.columns)

In [11]:
print("Train Size  : ", len(train_df))
print("Test Size : ", len (test_df))

Train Size  :  80668
Test Size :  20168


Since we performed the train/test randomly on the interactions, not all users and items may be present in the training set. We will relabel all of users and items to ensure the highest label is the number of users and items, respectively.

In [12]:
le_user = pp.LabelEncoder()
le_item = pp.LabelEncoder()
train_df['user_id_idx'] = le_user.fit_transform(train_df['userId'].values)
train_df['item_id_idx'] = le_item.fit_transform(train_df['movieId'].values)

In [13]:
train_user_ids = train_df['userId'].unique()
train_item_ids = train_df['movieId'].unique()

print(len(train_user_ids), len(train_item_ids))

test_df = test_df[
  (test_df['userId'].isin(train_user_ids)) & \
  (test_df['movieId'].isin(train_item_ids))
]
print(len(test))

610 8959
20168


In [14]:
# 这里是对 test_df 用同样的 transform? 我忘记原因是啥？
# "If you fit() to your test data, you'd compute a new mean and variance for each feature."
# https://stackoverflow.com/questions/48692500/fit-transform-on-training-data-and-transform-on-test-data
# 就是用 training set the statistics 来 transform test set 但是 不能重新 fit 一次 test set. Fit 是重新算 statistics (mean, vairance)

In [15]:
test_df['user_id_idx'] = le_user.transform(test_df['userId'].values)
test_df['item_id_idx'] = le_item.transform(test_df['movieId'].values)

In [16]:
n_users = train_df['user_id_idx'].nunique()
n_items = train_df['item_id_idx'].nunique()
print("Number of Unique Users : ", n_users)
print("Number of unique Items : ", n_items)


print(train_df['user_id_idx'].max())
print(train_df['item_id_idx'].max())
print(train_df['item_id_idx'].min())

Number of Unique Users :  610
Number of unique Items :  8959
609
8958
0


### Minibatch Sampling

Explain the scheme of minibatch positive and negative sample in some amount of prose.

We need to add `n_usr` to the sampled positive and negative items, since each node must have a unique id when using PyG.

In [17]:
# 这里有考虑 bipartite 对 edge_index 的影响吗？
def data_loader(data, batch_size, n_usr, n_itm):

    def sample_neg(x):
        while True:
            neg_id = random.randint(0, n_itm - 1)
            if neg_id not in x:
                return neg_id

    interected_items_df = data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()
    indices = [x for x in range(n_usr)]

    if n_usr < batch_size:
        users = [random.choice(indices) for _ in range(batch_size)]
    else:
        users = random.sample(indices, batch_size)
        
    users.sort()
    users_df = pd.DataFrame(users,columns = ['users'])

    interected_items_df = pd.merge(interected_items_df, users_df, how = 'right', left_on = 'user_id_idx', right_on = 'users')
    pos_items = interected_items_df['item_id_idx'].apply(lambda x : random.choice(x)).values
    neg_items = interected_items_df['item_id_idx'].apply(lambda x: sample_neg(x)).values

    return (
        torch.LongTensor(list(users)).to(device), 
        torch.LongTensor(list(pos_items)).to(device) + n_usr, 
        torch.LongTensor(list(neg_items)).to(device) + n_usr
    )

data_loader(train_df, 16, n_users, n_items)

(tensor([ 40,  92, 141, 190, 203, 271, 294, 317, 364, 372, 409, 430, 466, 530,
         567, 603]),
 tensor([4089, 1759,  937,  662, 7815, 6612, 1364, 3000, 9124,  729, 3662, 1073,
         2353, 1290, 2323,  751]),
 tensor([7175, 4834, 5409, 6325, 5558, 8328, 2864, 3942, 1735, 7013, 6413, 3481,
         1146, 8411, 4848, 4932]))

## Edge Index

PyG represents graphs as sparse lists of node pairs. Since our graph is undirected, we need to include each edge twice, once for the edges from the users to the items and vice-versa.

Similar to above, we add `n_users` to the item tensor to ensure that every node in the graph has a unique identifier.

In [18]:
#喜欢他这里的 bipartite 的处理，我之前的处理是转成 matrix 再转回来
# 他这里直接靠 offset， 但是他 eval 的时候，index 会不会有点问题?
u_t = torch.LongTensor(train_df.user_id_idx)
i_t = torch.LongTensor(train_df.item_id_idx) + n_users

train_edge_index = torch.stack((
  torch.cat([u_t, i_t]),
  torch.cat([i_t, u_t])
)).to(device)

# 这里应该就已经 做了 bipartite 处理了？因为 train_df size 是 80668
print(train_edge_index)
print(train_edge_index.size())

# 看一下 item id 有没有变化?
print()
print(train_edge_index.max())
print(train_edge_index.min())

# 看下这个, 有，609 + 8958 = 9568 
# 也就是这里确实是已经做了 bipartite 的 adj_mat 处理了
print()
print(i_t.max())
print(i_t.min())



tensor([[ 209,  293,  598,  ..., 2496, 1568, 7569],
        [9216, 1637, 1158,  ...,  325,  607,   17]])
torch.Size([2, 161336])

tensor(9568)
tensor(0)

tensor(9568)
tensor(610)


In [19]:
u_t_test = torch.LongTensor(test_df.user_id_idx)
i_t_test = torch.LongTensor(test_df.item_id_idx) + n_users

test_edge_index = torch.stack((
  torch.cat([u_t_test, i_t_test]),
  torch.cat([i_t_test, u_t_test])
)).to(device)


_--------------------- v--------------------_

In [ ]:
# check density
sparse_train_edge_index = SparseTensor(row=train_edge_index[0], 
                                       col=train_edge_index[1], 
                                       sparse_sizes=((n_users + n_items), (n_users + n_items)))

# 卧槽，怪不得这个 recall 这么搞，原版的能到 30% 尼玛 density 2.13， 我的那个版本只有 density=0.15%
print(sparse_train_edge_index)

_--------------------- ^ --------------------_

Let's confirm that the first and last edges match the middle two edges, but with the order of nodes swapped.

这里就是确认一下上面的 bipartite 处理是可以的

In [20]:
train_edge_index[:,-1], train_edge_index[:, 0]

(tensor([7569,   17]), tensor([ 209, 9216]))

In [21]:
# 这里 len(train) 是 bipartite 处理前的 length 所以就相当于中间
print(len(train))
train_edge_index[:, len(train)-1], train_edge_index[:, len(train)]

80668


(tensor([  17, 7569]), tensor([9216,  209]))

## Model Architecture

First, let's take a look at the graph convolutional layers that will power our recommender system GNN. Then, we can implement a wrapper to stack multiple convolutional layers.

### LightGCN Convolutional Layer

The LightGCN architecture is governed by the following rules:

$$e_{u}^{(k+1)} = \sum\limits_{i \in N_u} \frac{1}{\sqrt{|N_u|}\sqrt{|N_i|}}e^{(k)}_i$$

$$e_{i}^{(k+1)} = \sum\limits_{u \in N_i} \frac{1}{\sqrt{|N_i|}\sqrt{|N_u|}}e^{(k)}_u$$
In essence, the embedding for each node after a single LightGCN layer is the sum of the synthetic normalized embeddings of it's neighbors before the layer.


Briefly explain how the `MessagePassing` class works (look at colabs)

We can specify the type of aggregation our `MessagePassing` layer should use by passing in an `aggr=` argument in the layer initialization. Here we use `add` to specify summation aggregation of messages.

Note that we could have manually defined our aggregation function by defining a function explicitly in the class:
```
def aggregate(self, x, messages, index):
  return torch_scatter.scatter(messages, index, self.node_dim, reduce="sum")
```
The `torch_scatter.scatter` function enables us to aggregate messages being sent to the same node. The `reduce=` argument specifies how to aggregate, while `index` has the same length as the `messages` tensor and maps from message to destination node.

哇，他这个写的好精简呀, 而且是比较标准，没有用 gcn_noam 和 sparse tensor

In [22]:
class LightGCNConv(MessagePassing):
  def __init__(self, **kwargs):  
    super().__init__(aggr='add')

  def forward(self, x, edge_index):
    # Compute normalization
    # 啊，原来 GCN norm 的计算是这么写的, 懂了
    from_, to_ = edge_index
    deg = degree(to_, x.size(0), dtype=x.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

    # Start propagating messages (no update after aggregation)
    return self.propagate(edge_index, x=x, norm=norm)

  def message(self, x_j, norm):
    return norm.view(-1, 1) * x_j

Let's test out our implementation of the LightGCN convolution by applying it to a small bipartite graph. 

This sample graph is undirected, and node 0 is connected to nodes 2 and 3 while node 1 is connected to 3 and 4.

In [23]:
test_x = torch.Tensor(np.eye(5))
test_edge_index = torch.LongTensor(np.array([
  [0, 0, 1, 1, 2, 3, 3, 4],
  [2, 3, 3, 4, 0, 0, 1, 1]
]))

LightGCNConv()(test_x, test_edge_index)

tensor([[0.0000, 0.0000, 0.7071, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5000, 0.7071],
        [0.7071, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.7071, 0.0000, 0.0000, 0.0000]])

Notice how each node has an initial feature vector corresponding to a one-hot encoding at the index of their id.

As we expected, node 0 received messages (and so has non-zero features at the corresponding indicies) from nodes 2 and 3. We can easily verify that nodes 1, 2, 3, and 4 also received messages from their precisely neighbors.

### NGCF Layer

NGCF is an older architecture than LightGCN that originated by researchers who applied [Graph Convolutional Networks (GCNs)]() to recommender systems. LightGCN functions the same as NGCF, but removes the learnable linear layers, non-linear activation, and dropout.

One layer of NGCF updates user and item embeddings as follows:

$$e_{u}^{(k+1)} = \sigma\left(W_1 e_u^{(k)} + \sum\limits_{i \in N_u} \frac{1}{\sqrt{|N_u|}\sqrt{|N_i|}}(W_1e^{(k)}_i + W_2(e^{(k)}_i \odot e^{(k)}_u))\right)$$

$$e_{i}^{(k+1)} = \sigma\left(W_1 e_i^{(k)} + \sum\limits_{u \in N_i} \frac{1}{\sqrt{|N_i|}\sqrt{|N_u|}}(W_1e^{(k)}_u + W_2(e^{(k)}_u \odot e^{(k)}_i))\right)$$

Typically, NGCF is implemented with dropout before the activation and with an activation function $\sigma$ of LeakyReLU.

In [24]:
class NGCFConv(MessagePassing):
  def __init__(self, latent_dim, dropout, bias=True, **kwargs):  
    super(NGCFConv, self).__init__(aggr='add', **kwargs)

    self.dropout = dropout

    self.lin_1 = nn.Linear(latent_dim, latent_dim, bias=bias)
    self.lin_2 = nn.Linear(latent_dim, latent_dim, bias=bias)

    self.init_parameters()


  def init_parameters(self):
    nn.init.xavier_uniform_(self.lin_1.weight)
    nn.init.xavier_uniform_(self.lin_2.weight)


  def forward(self, x, edge_index):
    # Compute normalization
    from_, to_ = edge_index
    deg = degree(to_, x.size(0), dtype=x.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

    # Start propagating messages
    out = self.propagate(edge_index, x=(x, x), norm=norm)

    # Perform update after aggregation
    out += self.lin_1(x)
    out = F.dropout(out, self.dropout, self.training)
    return F.leaky_relu(out)


  def message(self, x_j, x_i, norm):
    return norm.view(-1, 1) * (self.lin_1(x_j) + self.lin_2(x_j * x_i))

### Recommender System GNN

For this tutorial, we will be using the following class, `RecSysGNN` in order to stack the NGCF or LightGCN convolutional layers. Some considerations that can be made for tweaking the models are the number of layers of your model and dropout. The more number of layers you add to the model, the more your model will "diffuse" information of recommendations made from nodes that are `n`-hops away in a model that uses `n` layers. Dropout can be tweaked to try out different regularization schemes.

Notice that our forward function works differently from most neural networks by forward propagating embeddings for all nodes in the graph. This is because the embeddings for a given node depend on the embeddings of it's `n`-hop neighborhood, so they need to be propagated as well.

In [25]:
class RecSysGNN(nn.Module):
  def __init__(
      self,
      latent_dim, 
      num_layers,
      num_users,
      num_items,
      model, # 'NGCF' or 'LightGCN'
      dropout=0.1 # Only used in NGCF
  ):
    super(RecSysGNN, self).__init__()

    assert (model == 'NGCF' or model == 'LightGCN'), 'Model must be NGCF or LightGCN'
    self.model = model
    self.embedding = nn.Embedding(num_users + num_items, latent_dim) # latent_dim 这个就是 vector len 貌似都是用 64

    if self.model == 'NGCF':
      self.convs = nn.ModuleList(
        NGCFConv(latent_dim, dropout=dropout) for _ in range(num_layers)
      )
    else:
      self.convs = nn.ModuleList(LightGCNConv() for _ in range(num_layers))

    self.init_parameters()


  def init_parameters(self):
    if self.model == 'NGCF':
      nn.init.xavier_uniform_(self.embedding.weight, gain=1)
    else:
      # Authors of LightGCN report higher results with normal initialization
      nn.init.normal_(self.embedding.weight, std=0.1) 


  # 看下这里有没有对 bipartite adj mat edge_index 做别的处理? 
  # 没有，就是标准写法
  def forward(self, edge_index):
    emb0 = self.embedding.weight
    embs = [emb0]

    emb = emb0
    for conv in self.convs:
      emb = conv(x=emb, edge_index=edge_index)
      embs.append(emb)

    out = (
      torch.cat(embs, dim=-1) if self.model == 'NGCF' 
      else torch.mean(torch.stack(embs, dim=0), dim=0)
    )
    
    return emb0, out


  def encode_minibatch(self, users, pos_items, neg_items, edge_index):
    # 拿 bipartite adj mat 处理过的 edge_index 自己过一遍，得到 emb0 和 emb_final
    emb0, out = self(edge_index)
    return (
        out[users], 
        out[pos_items], 
        out[neg_items], 
        emb0[users],
        emb0[pos_items],
        emb0[neg_items]
    )

## Loss function and metrics

We implement both the Bayesian Personalized Ranking loss function for a single minibatch of users, positive items, and negative items, as well as the precision@K and recall@K metrics.

In [26]:
def compute_bpr_loss(users, users_emb, pos_emb, neg_emb, user_emb0,  pos_emb0, neg_emb0):
    # compute loss from initial embeddings, used for regulization
    reg_loss = (1 / 2) * (
    user_emb0.norm().pow(2) + 
    pos_emb0.norm().pow(2)  +
    neg_emb0.norm().pow(2)
    ) / float(len(users))

    # compute BPR loss from user, positive item, and negative item embeddings
    pos_scores = torch.mul(users_emb, pos_emb).sum(dim=1)
    neg_scores = torch.mul(users_emb, neg_emb).sum(dim=1)

    bpr_loss = torch.mean(F.softplus(neg_scores - pos_scores))

    return bpr_loss, reg_loss

In [27]:
# 下面这个估计得 explore 一下？

In [28]:
# 这里完全没有用到 train_data.. 
def get_metrics(user_Embed_wts, item_Embed_wts, n_users, n_items, train_data, test_data, K):
  
    test_user_ids = torch.LongTensor(test_data['user_id_idx'].unique())
  
    # compute the score of all user-item pairs
    relevance_score = torch.matmul(user_Embed_wts, torch.transpose(item_Embed_wts,0, 1))

    # create dense tensor of all user-item interactions
    # 这是啥？
    i = torch.stack((
        torch.LongTensor(train_df['user_id_idx'].values),
        torch.LongTensor(train_df['item_id_idx'].values)
    ))
    
    # 这里居然直接用的是 train_df??
    v = torch.ones((len(train_df)), dtype=torch.float64)
    
    # 这个是咋弄的? 哦, sparse tensor 就是按照 COO 格式来 init， 诶，那怎么 v 全部用 1 呢？？
    # 下面有一个 (1 - interactions_t)?? 这里有点乱其实.. 这个是为了 masking?? 因为上面是  train data
    # 这边还是没有看到 bipartite 和 train / test data 的 index 的影响, 我觉得大概率是直接减？
    # ?? 这里 n_items 是 train data 不是全部的 n_items...
    interactions_t = torch.sparse.FloatTensor(i, v, (n_users, n_items)).to_dense().to(device)

    # mask out training user-item interactions from metric computation
    relevance_score = torch.mul(relevance_score, (1 - interactions_t))

    # compute top scoring items for each user
    topk_relevance_indices = torch.topk(relevance_score, K).indices
    topk_relevance_indices_df = pd.DataFrame(topk_relevance_indices.cpu().numpy() ,columns =['top_indx_'+str(x+1) for x in range(K)])
    topk_relevance_indices_df['user_ID'] = topk_relevance_indices_df.index
    topk_relevance_indices_df['top_rlvnt_itm'] = topk_relevance_indices_df[['top_indx_'+str(x+1) for x in range(K)]].values.tolist()
    topk_relevance_indices_df = topk_relevance_indices_df[['user_ID','top_rlvnt_itm']]

    # measure overlap between recommended (top-scoring) and held-out user-item 
    # interactions
    test_interacted_items = test_data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()
    metrics_df = pd.merge(test_interacted_items,topk_relevance_indices_df, how= 'left', left_on = 'user_id_idx',right_on = ['user_ID'])
    metrics_df['intrsctn_itm'] = [list(set(a).intersection(b)) for a, b in zip(metrics_df.item_id_idx, metrics_df.top_rlvnt_itm)]

    metrics_df['recall'] = metrics_df.apply(lambda x : len(x['intrsctn_itm'])/len(x['item_id_idx']), axis = 1) 
    metrics_df['precision'] = metrics_df.apply(lambda x : len(x['intrsctn_itm'])/K, axis = 1)

    return metrics_df['recall'].mean(), metrics_df['precision'].mean()

## Train and evaluate models

Now that we've implemented both LightGCN and NGCF in PyG, we can train and evaluate their performance!

In [29]:
latent_dim = 64
n_layers = 3 

EPOCHS = 50
BATCH_SIZE = 1024
DECAY = 0.0001
LR = 0.005 
K = 20

In [30]:
# convert bipartite adj mat based edge index to interaction mat based edge_index
# 跟上面的是反的，就是为了方便 算 BPR 和 recall 的时候，可以 semantically  用 row0 是 user, row1 是 item
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index):
    # 得先定义一个 sparse 不然没法转 dense matrix
    sparse_input_edge_index = SparseTensor(row=input_edge_index[0], 
                                           col=input_edge_index[1], 
                                           sparse_sizes=((num_users + num_movies), num_users + num_movies))
    adj_mat = sparse_input_edge_index.to_dense()
    interact_mat = adj_mat[: num_users, num_users :]
    r_mat_edge_index = interact_mat.to_sparse_coo().indices()
    return r_mat_edge_index

In [31]:
r_mat_train_edge_index = convert_adj_mat_edge_index_to_r_mat_edge_index(train_edge_index)

In [32]:
# 看下这里有没有什么特殊处理对 bipartite adj mat egdge list?
# 这里传的是 train_df 不是 train_edge_index?? 看下下面怎么弄？？
def train_and_eval(model, optimizer, train_df):
  loss_list_epoch = []
  bpr_loss_list_epoch = []
  reg_loss_list_epoch = []

  recall_list = []
  precision_list = []

  for epoch in tqdm(range(EPOCHS)):
      n_batch = int(len(train)/BATCH_SIZE)
    
      final_loss_list = []
      bpr_loss_list = []
      reg_loss_list = []
    
      model.train()
      for batch_idx in range(n_batch):

          optimizer.zero_grad()

          # ok, 这里是用 train_df 也就是没有 bipartite adj mat 处理的, 但是得到 user_item pair..  
          users, pos_items, neg_items = data_loader(train_df, BATCH_SIZE, n_users, n_items)
          
          # 这里才是去拿到 embedding， 这里用的，就是 bipartite 处理过的 train_edge_index 了  
          # 也就是说他这里 bpr loss 也是跟我一样用 bipartite 处理过的 train_edge_index 来弄
          users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0 = model.encode_minibatch(users, 
                                                                                            pos_items, 
                                                                                            neg_items, 
                                                                                            train_edge_index)

          bpr_loss, reg_loss = compute_bpr_loss(users, 
                                                users_emb, 
                                                pos_emb, 
                                                neg_emb, 
                                                userEmb0,  
                                                posEmb0, 
                                                negEmb0)
        
          reg_loss = DECAY * reg_loss
          final_loss = bpr_loss + reg_loss

          final_loss.backward()
          optimizer.step()

          final_loss_list.append(final_loss.item())
          bpr_loss_list.append(bpr_loss.item())
          reg_loss_list.append(reg_loss.item())

      model.eval()
      with torch.no_grad():
          _, out = model(train_edge_index)
          final_user_Embed, final_item_Embed = torch.split(out, (n_users, n_items))
          
          
          test_topK_recall,  test_topK_precision = get_metrics(final_user_Embed, 
                                                               final_item_Embed, 
                                                               n_users, 
                                                               n_items, 
                                                               train_df, 
                                                               test_df, 
                                                               K
          )
              
            
          print(f"epoch-batch_idx: {epoch}-{batch_idx}: train_loss: {final_loss.item()}   test_topK_recall {test_topK_recall}, test_topK_precision {test_topK_precision}")

      loss_list_epoch.append(round(np.mean(final_loss_list),4))
      bpr_loss_list_epoch.append(round(np.mean(bpr_loss_list),4))
      reg_loss_list_epoch.append(round(np.mean(reg_loss_list),4))

      recall_list.append(round(test_topK_recall,4))
      precision_list.append(round(test_topK_precision,4))

  return (
    loss_list_epoch, 
    bpr_loss_list_epoch, 
    reg_loss_list_epoch, 
    recall_list, 
    precision_list
  )

### Train and eval LightGCN

In [33]:
print(f"n_users: {n_users}, n_items {n_items}")

n_users: 610, n_items 8959


In [34]:
lightgcn = RecSysGNN(
    latent_dim=latent_dim, 
    num_layers=n_layers,
    num_users=n_users,
    num_items=n_items,
    model='LightGCN'
)
lightgcn.to(device)

optimizer = torch.optim.Adam(lightgcn.parameters(), lr=LR)
print("Size of Learnable Embedding : ", [x.shape for x in list(lightgcn.parameters())])

Size of Learnable Embedding :  [torch.Size([9569, 64])]


In [ ]:
light_loss, light_bpr, light_reg, light_recall, light_precision = train_and_eval(lightgcn, optimizer, train_df)

  0%|          | 0/50 [00:00<?, ?it/s]

epoch-batch_idx: 0-77: train_loss: 0.24155192077159882   test_topK_recall 0.1347642946693434, test_topK_precision 0.13565573770491804
epoch-batch_idx: 1-77: train_loss: 0.20748399198055267   test_topK_recall 0.1512197823357172, test_topK_precision 0.1483606557377049
epoch-batch_idx: 2-77: train_loss: 0.18385258316993713   test_topK_recall 0.17168730371888805, test_topK_precision 0.16303278688524592
epoch-batch_idx: 3-77: train_loss: 0.1607300490140915   test_topK_recall 0.17566346171070366, test_topK_precision 0.16459016393442624
epoch-batch_idx: 4-77: train_loss: 0.17203058302402496   test_topK_recall 0.17420862291712214, test_topK_precision 0.16311475409836065
epoch-batch_idx: 5-77: train_loss: 0.15099895000457764   test_topK_recall 0.17733222136624502, test_topK_precision 0.1648360655737705
epoch-batch_idx: 6-77: train_loss: 0.11276644468307495   test_topK_recall 0.17940750947972586, test_topK_precision 0.16696721311475413
epoch-batch_idx: 7-77: train_loss: 0.11198142170906067   tes

In [ ]:
epoch_list = [(i+1) for i in range(EPOCHS)]

In [ ]:
plt.plot(epoch_list, light_loss, label='Total Training Loss')
plt.plot(epoch_list, light_bpr, label='BPR Training Loss')
plt.plot(epoch_list, light_reg, label='Reg Training Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.plot(epoch_list, light_recall, label='Recall')
plt.plot(epoch_list, light_precision, label='Precision')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.legend()

### Train and eval NGCF

In [ ]:
ngcf = RecSysGNN(
    latent_dim=latent_dim, 
    num_layers=n_layers,
    num_users=n_users,
    num_items=n_items,
    model='NGCF'
)
ngcf.to(device)

optimizer = torch.optim.Adam(ngcf.parameters(), lr=LR)
print("Size of Learnable Embedding : ", [x.shape for x in list(ngcf.parameters())])

In [ ]:
ngcf_loss, ngcf_bpr, ngcf_reg, ngcf_recall, ngcf_precision = train_and_eval(ngcf, optimizer, train_df)

In [ ]:
epoch_list = [(i+1) for i in range(EPOCHS)]

In [ ]:
plt.plot(epoch_list, ngcf_loss, label='Total Training Loss')
plt.plot(epoch_list, ngcf_bpr, label='BPR Training Loss')
plt.plot(epoch_list, ngcf_reg, label='Reg Training Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.plot(epoch_list, ngcf_recall, label='Recall')
plt.plot(epoch_list, ngcf_precision, label='Precision')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.legend()

### Compare model performance

In [ ]:
max(light_precision), max(light_recall)

In [ ]:
max(ngcf_precision), max(ngcf_recall)

## Paper References

1. Harper, F. Maxwell, and Konstan, Joseph A. “The MovieLensDatasets: History and Context.” ACM Transactions on Interactive Intelligence Systems (TiiS) 5, 4. 2015.
2. He, Xiangnan, et al. “LightGCN: Simplifying and powering graph convolution network for recommendation.” Proceedings of the 43rd International ACM SIGIR Conference on Research and Development in Information Retrieval. 2020.
3. Wang, Xiang, et al. “Neural graph collaborative filtering.” Proceedings of the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval. 2019.

## Code References

We thank the authors of the following codebases and notebooks, from which parts of this tutorial were inspired or adapted.

- https://www.kaggle.com/dipanjandas96/lightgcn-pytorch-from-scratch

- https://github.com/gusye1234/LightGCN-PyTorch

- https://github.com/SytzeAndr/NGCF_RP32/blob/hand-in/NGCF.ipynb
